In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html
!pip install wandb
!wandb login

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 220.5 MB 43 kB/s 
     |████████████████████████████████| 77 kB 2.8 MB/s 
     |████████████████████████████████| 281 kB 22.1 MB/s 
     |████████████████████████████████| 51 kB 532 kB/s 
     |████████████████████████████████| 45 kB 3.9 MB/s 
     |████████████████████████████████| 109 kB 60.7 MB/s 
     |████████████████████████████████| 596 kB 65.2 MB/s 
     |████████████████████████████████| 11.1 MB 69.1 MB/s 
     |████████████████████████████████| 103 kB 89.7 MB/s 
     |████████████████████████████████| 42 kB 916 kB/s 
     |████████████████████████████████| 3.1 MB 78.8 MB/s 
     |████████████████████████████████| 546 kB 78.3 MB/s 
     |████████████████████████████████| 121 kB 75.3 MB/s 
     |████████████████████████████████| 90 kB 11.1 MB/s 
     |██████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.2 MB/s 
     |████████████████████████████████| 145 kB 66.6 MB/s 
     |████████████████████████████████| 181 kB 60.5 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=7018a763b4c1f54b68f472436bdcca9b944c671a32186a47721189fdbb80b873
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
%ls
%load_ext autoreload
%autoreload 2

drive/  sample_data/


In [4]:
%cd drive
%cd MyDrive
%cd SIG-VAE-GIN-and-fMRI
%cd SIG-VAE-GIN-and-fMRI
from sigvaemodel import *
from loss import *

[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI
[Errno 2] No such file or directory: 'MyDrive'
/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI
/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI/SIG-VAE-GIN-and-fMRI
[Errno 2] No such file or directory: 'SIG-VAE-GIN-and-fMRI'
/content/drive/MyDrive/SIG-VAE-GIN-and-fMRI/SIG-VAE-GIN-and-fMRI


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [5]:
%reload_ext autoreload
from sigvaemodel import *
from loss import loss as loss_function
from loss import get_rec

In [6]:
import numpy as np
import torch
import torch.nn as nn
import dgl.data
import networkx as nx
import scipy.sparse as sp
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
import sigvaemodel
import loss
import time
import wandb
from matplotlib import pyplot


CoraDataset=dgl.data.CoraGraphDataset()
graph=CoraDataset[0]
featureMatrix=graph.ndata['feat']
featureMatrix=featureMatrix.view(1, featureMatrix.shape[0], featureMatrix.shape[1])
print(featureMatrix.size())
print(featureMatrix)
graph.to('cuda')
config=dict(
    learning_rate=0.0001,
    architecture="GIN",
    dataset="Cora",
    noise_dim=32,
    warming_up=100,
)


Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
torch.Size([1, 2708, 1433])
tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]]])


In [7]:
def edges_to_lists(headMatrix, tailMatrix):
    results=[]
    assert len(headMatrix)==len(tailMatrix)
    hlength=len(headMatrix)
    for i in range(hlength):
        results.append([headMatrix[i], tailMatrix[i]])
    return results

In [8]:
def preprocess(graph):
    N=graph.num_nodes()
    u, v=graph.edges()
    numE=graph.number_of_edges()
    testidx=np.arange(graph.number_of_edges())
    testidx=np.random.permutation(testidx)
    testSize=int(len(testidx)/10)
    validation_size=int(len(testidx)/5)
    train_size=graph.number_of_edges()-testSize-validation_size
    
    test_pos_u, test_pos_v=u[testidx[:testSize]], v[testidx[:testSize]]
    train_pos_u, train_pos_v = u[testidx[testSize:numE-validation_size]], v[testidx[testSize:numE-validation_size]]
    
    validation_pos_u, validation_pos_v=u[testidx[numE-validation_size:]], v[testidx[numE-validation_size:]]
    
    adj=sp.coo_matrix((np.ones(len(u)), (u.detach().cpu().numpy(), v.detach().cpu().numpy())))
    adj_neg=1-adj.todense()-np.eye(graph.number_of_nodes())
    adj=torch.Tensor(adj.todense())
    neg_u, neg_v=np.where(adj_neg!=0)
    #print(len(neg_u))
    #print(len(neg_v))
    neg_testidx=np.random.choice(len(neg_u), graph.number_of_edges())
    test_neg_u, test_neg_v=neg_u[neg_testidx[:testSize]], neg_v[neg_testidx[:testSize]]
    train_neg_u, train_neg_v=neg_u[neg_testidx[testSize:numE-validation_size]], neg_v[neg_testidx[testSize:numE-validation_size]]
    validation_neg_u, validation_neg_v=neg_u[neg_testidx[numE-validation_size:]], neg_v[neg_testidx[numE-validation_size:]]
    #print(test_neg_u)
    #print(test_neg_v)
    adj_train=sp.coo_matrix((np.ones(len(train_pos_u)), (train_pos_u.detach().cpu().numpy(), train_pos_v.detach().cpu().numpy())),shape=(N, N))
    adj_train=adj_train-sp.dia_matrix((adj_train.diagonal()[np.newaxis, :], [0]), shape=adj_train.shape)
    adj_train=torch.Tensor(adj_train.todense())
    
    adj_test=sp.coo_matrix((np.ones(len(test_pos_u)), (test_pos_u.detach().cpu().numpy(), test_pos_v.detach().cpu().numpy())),shape=(N,N))
    adj_test=adj_test-sp.dia_matrix((adj_test.diagonal()[np.newaxis, :], [0]), shape=adj_test.shape)
    adj_test=torch.Tensor(adj_test.todense())

    adj_validation=sp.coo_matrix((np.ones(len(validation_pos_u)), (validation_pos_u.detach().cpu().numpy(), validation_pos_v.detach().cpu().numpy())),shape=(N,N))
    adj_validation=adj_validation-sp.dia_matrix((adj_validation.diagonal()[np.newaxis, :], [0]), shape=adj_validation.shape)
    adj_validation=torch.Tensor(adj_validation.todense())

    train_graph=dgl.remove_edges(graph, np.hstack((testidx[:testSize],testidx[numE-validation_size:])))
    test_graph=dgl.remove_edges(graph, testidx[testSize:])
    validation_graph=dgl.remove_edges(graph, testidx[:numE-validation_size]) 

    train_pos=edges_to_lists(train_pos_u, train_pos_v)
    test_pos=edges_to_lists(test_pos_u, test_pos_v)
    test_neg=edges_to_lists(test_neg_u, test_neg_v)
    train_neg=edges_to_lists(train_neg_u, train_neg_v)
    validation_pos=edges_to_lists(validation_pos_u, validation_pos_v)
    validation_neg=edges_to_lists(validation_neg_u, validation_neg_v)

    featureMatrix=graph.ndata['feat']
    featureMatrix=featureMatrix.view(1, featureMatrix.shape[0], featureMatrix.shape[1]).to('cuda')


    return adj, adj_train, adj_validation, adj_test, featureMatrix, train_graph, validation_graph, test_graph, train_pos, validation_pos, test_pos, train_neg, validation_neg, test_neg


In [9]:
def step(model, adj, adj_label, feat, input_graph, epoch, optimizer, weight, norm, device):
  #global loss
    if optimizer is None:
        model.eval()
    else:
        model.train()
    optimizer.zero_grad()
    if optimizer is None:
      generated_prob, mu, sigma, latent_representation, Z, epsilon=model.forward(adj.to(device), feat.to(device), input_graph)
    else:
      #N=adj.shape[0]
      generated_prob, mu, sigma, latent_representation, Z, epsilon=model.forward(adj.to(device), feat.to(device), input_graph)
      loss_rec, loss_prior, loss_post=loss_function(generated_prob, adj_label, mu, sigma, Z, epsilon, latent_representation, norm, weight, device=device)
      reg=(loss_post-loss_prior) / (adj.size(0)**2)
      loss=loss_rec+reg
    #print(generated_prob.shape)
    #print(adj.shape)
    #loss=loss_rec+loss_prior+loss_post
    
    #WarmingUP=np.min([epoch/300,1])
    #reg=(loss_post-loss_prior) * WarmingUP / (adj.size(0)**2)
    #loss=loss_rec+WarmingUP*reg
    
    
    if optimizer is not None:
        loss.backward()
        optimizer.step()
    return latent_representation, generated_prob, loss, Z

In [10]:
def get_auc(embedding, pos_array, neg_array, decoder_type):
    def dc(x):
      if decoder_type=="ip":
        return 1/(1+torch.exp(-x))
      elif decoder_type=="bp":
        return 1-torch.exp(-torch.exp(x))
    #print("92499092409")
    #print(embedding)
    #print(pos_array)
    #print(neg_array)
    N=embedding.shape[0]
    emb_pos_head=embedding[:, pos_array[0], :]
    emb_pos_tail=embedding[:, pos_array[1], :]
    preds_pos=dc(torch.einsum('abc, abc->ab', emb_pos_head, emb_pos_tail))
    #preds_pos=torch.einsum('abc, abc->ab', emb_pos_head, emb_pos_tail)
    #emb_neg_head=torch.gather(embedding, 1, neg_array[0].view(-1,1))
    #emb_neg_tail=torch.gather(embedding, 1, neg_array[1].view(-1,1))
    emb_neg_head=embedding[:, neg_array[0], :]
    emb_neg_tail=embedding[:, neg_array[1], :]
    preds_neg=dc(torch.einsum('abc, abc->ab', emb_neg_head, emb_neg_tail))
    #preds_neg=torch.einsum('abc, abc->ab', emb_neg_head, emb_neg_tail)
    preds=torch.hstack([preds_pos, preds_neg]).detach().cpu().numpy()
    #print(preds)
    labels=torch.hstack([torch.ones(preds_pos.shape[-1]), torch.zeros(preds_neg.shape[-1])]).detach().cpu().numpy()
    
    roc=torch.Tensor([roc_auc_score(labels, pred.flatten()) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    fpr_accumulate=0
    recall_accumulate=0
    precision_accumulate=0
    for pred in np.vsplit(preds,N):
      fpr, recall, _=roc_curve(labels, pred.flatten())
      precision, _, _=precision_recall_curve(labels, pred.flatten())
      fpr_accumulate+=fpr.mean()
      recall_accumulate+=recall.mean()
      precision_accumulate+=precision.mean()
    fpr_mean=fpr_accumulate/N
    recall_mean=recall_accumulate/N
    precision_mean=precision_accumulate/N
    
    #precision, recall=torch.Tensor([precision_recall_curve(labels, pred.flatten()) for pred in np.vsplit(preds, embedding.shape[0])])
    #accuracy=torch.Tensor([accuracy_score(labels, pred.flatten()) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    #precision=torch.Tensor([precision_score(labels, pred) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    #recall=torch.Tensor([recall_score(labels, pred) for pred in np.vsplit(preds, embedding.shape[0])]).mean()
    #precision=precision.mean()
    #recall=recall.mean()
    #fpr=fpr.mean()
    return roc, precision_mean, recall_mean, fpr_mean

In [ ]:
if torch.cuda.is_available():
  device='cuda'
else:
  device='cpu'

import wandb

wandb.config = {
  "learning_rate": 0.00001,
  "epochs": 3000,
  "batch_size": 35,
  "K" : 15,
  "J" : 20, 
  "noise_dim" : 32
}
#wandb.init(project="SIG-VAE-GIN-and-fMRI", entity="ytrewq271828", config=config)

adj, adj_train, adj_validation, adj_test, featureMatrix, train_graph,validation_graph, test_graph, train_pos, validation_pos, test_pos, train_neg, validation_neg, test_neg=preprocess(graph)
adj_label=adj_train+torch.eye(adj_train.shape[0])
adj_label=torch.FloatTensor(adj_label)

adj=torch.Tensor(adj).to(device)
adj_test=torch.Tensor(adj_test).to(device)
featureMatrix=torch.cuda.FloatTensor(featureMatrix)
adj_train=torch.Tensor(adj_train).to(device)
train_graph=train_graph.to(device)
test_graph=test_graph.to(device)
validation_graph=validation_graph.to(device)
graph=graph.to(device)
train_graph=train_graph.to(device)
test_graph=test_graph.to(device)
validation_graph=validation_graph.to(device)
N=adj.shape[0]
weight=torch.tensor([float(N * N - adj.sum()) / adj.sum()]).to(device)
norm=N*N/float((N*N-adj.sum())*2)

pos_array_train=torch.cuda.LongTensor(train_pos).t() #(2, #positive edges)
neg_array_train=torch.cuda.LongTensor(train_neg).t() #(2, #negative edges)
pos_array_test=torch.cuda.LongTensor(test_pos).t()
neg_array_test=torch.cuda.LongTensor(test_neg).t()
pos_array_validation=torch.cuda.LongTensor(validation_pos).t()
neg_array_validation=torch.cuda.LongTensor(validation_pos).t()
#print(torch.count_nonzero(adj_train))
#print(torch.count_nonzero(adj_test))
#print(adj_test.shape)
#print(pos_array_train)

torch.autograd.set_detect_anomaly(False)

model=SIGVAE_GIN(Lu=1, 
                Lmu=1, 
                Lsigma=1, 
                input_dim=featureMatrix.shape[2], 
                output_dim_u=[32], 
                output_dim_mu=[16], 
                output_dim_sigma=[16],
                K=15,
                J=20,
                device=device,
                noise_dim=32,
                decoder_type="bp",
                activation=torch.nn.functional.relu,
                dropout=0)
optimizer=torch.optim.Adam(model.parameters(), lr=0.00001)
epoch=300
loss_accumulate=0
for epc in range(epoch):
    t=time.time()
    model.train()
    latent_representation, generated_prob, loss_train, z_proc=step(model, adj, adj_label, featureMatrix, graph,  epc, optimizer, weight, norm, device=device)
    loss_accumulate+=loss_train
    #print(z_proc)    
    #print(z_proc.shape)
    #z_proc=z_proc.detach().cpu().numpy()
    roc, precision, recall, fpr=get_auc(z_proc, pos_array_validation, neg_array_validation, decoder_type="bp")
    #roc, acc, prc, rcl=get_auc(z_proc, pos_array_train, neg_array_train, decoder_type="bp")
    #print(loss_train.item())
    #print(roc)
    print("Epoch : ", '%d' %(epc+1), "loss=", "%f" %(loss_train.item()),
          "time=", "%f" %(time.time()-t), "roc=", "%f" %(roc), 
          "precision", "%f" %(precision), "recall", "%f" %(recall),
          "false positive rate", "%f" %(fpr))
    #pyplot.plot(fpr, recall, marker='.', label='roc-auc')
    
    '''
    wandb.log({"loss_train": loss_train})
    wandb.log({"precision_train" : precision})
    wandb.log({"recall_train" : recall})
    wandb.log({"fpr_train" : fpr})
    wandb.watch(model)
    '''
    if (epc+1) % 20 == 0:
            model.eval()
            #generated_prob, mu, sigma, latent_representation, z_proc, epsilon=model(adj_test, featureMatrix, test_graph)
            latent_representation, generated_prob, loss_test, z_proc=step(model, adj, adj_label, featureMatrix, graph,  epc, optimizer, weight, norm, device=device)
            #z_proc = z_proc.detach().cpu().numpy()
            roc, _, _, _= get_auc(z_proc, pos_array_test, neg_array_test, decoder_type="bp")
            rslt = "Test ROC score: %f" %(roc)
            print("\n", rslt, "\n")
            '''
            wandb.log({"loss_test": loss_test})
            wandb.log({"roc-auc_test" : roc})
            wandb.watch(model)
            '''
    del latent_representation
    del generated_prob
    del z_proc
    del roc
    
    torch.cuda.empty_cache()
#pyplot.show()
    #latent_representation, generated_prob, loss_train=step(model, adj_train, epoch, optimizer )

Epoch :  1 loss= 365.646393 time= 2.617741 roc= 0.500000 precision 0.500289 recall 0.527779 false positive rate 0.527779
Epoch :  2 loss= 365.650909 time= 1.425818 roc= 0.500000 precision 0.500289 recall 0.529724 false positive rate 0.529724
Epoch :  3 loss= 365.724762 time= 1.422990 roc= 0.500000 precision 0.500291 recall 0.529183 false positive rate 0.529183
Epoch :  4 loss= 365.708130 time= 1.410701 roc= 0.500000 precision 0.500289 recall 0.533349 false positive rate 0.533349
Epoch :  5 loss= 365.597382 time= 1.419166 roc= 0.500000 precision 0.500290 recall 0.530668 false positive rate 0.530668
Epoch :  6 loss= 365.556488 time= 1.420775 roc= 0.500000 precision 0.500289 recall 0.532760 false positive rate 0.532760
Epoch :  7 loss= 365.603668 time= 1.429403 roc= 0.500000 precision 0.500289 recall 0.527010 false positive rate 0.527010
Epoch :  8 loss= 365.500275 time= 1.423044 roc= 0.500000 precision 0.500289 recall 0.533163 false positive rate 0.533163
Epoch :  9 loss= 365.407623 time

In [ ]:
  !nvidia-smi

Mon May 23 04:04:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    27W /  70W |  11124MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------